# Prediction

## Description

Using the model we created in the `3_Train_fit_evaluate_classifier.ipynb`, this notebook will make predictions on new data to generate a cropland mask for Eastern Africa. The notebook will ceate both pixel-wise classifications and classification probabilities. Results are saved to disk as Cloud-Optimised-Geotiffs.

1. Open and inspect the shapefile which delineates the extent we're classifying
2. Import the model
3. Make predictions on new data loaded through the ODC.  The pixel classification will also undergo a post-processing step where steep slopes and water are masked using a SRTM derivative and WOfS, respectively. Pixels labelled as crop above 3600 metres ASL are also masked. 

***
## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load Packages

In [ ]:
!pip install ../../production/cm_tools

In [1]:
import warnings
import os
import datacube
import numpy as np
import xarray as xr
import geopandas as gpd
import subprocess as sp
from joblib import load
from datacube.utils import geometry
from datacube.utils.cog import write_cog
from datacube.utils.geometry import assign_crs
from datacube.testutils.io import rio_slurp_xarray

from deafrica_tools.classification import HiddenPrints, predict_xr
from deafrica_tools.plotting import map_shapefile
from deafrica_tools.spatial import xr_rasterize

from cm_tools.africa_geobox import AfricaGeobox
from deafrica_tools.dask import create_local_dask_cluster
#import out feature layer function for prediction
from feature_layer_functions import gm_mads_two_seasons_prediction
from post_processing import post_processing

In [2]:
warnings.filterwarnings("ignore")

### Set up dask cluster

In [3]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:43453 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 31 Memory: 254.70 GB


## Analysis parameters

* `model_path`: The path to the location where the model exported from the previous notebook is stored
* `training_data`: Name and location of the training data `.txt` file output from runnning `1_Extract_training_data.ipynb`
* `test_shapefile`: A shapefile containing polygons that represent regions where you want to test your model. The shapefile should have a unique identifier as this will be used to export classification results to disk as geotiffs. Alternatively, this could be a shapefile that defines the extent of the entire AOI you want to classify.
* `results`: A folder location to store the classified geotiffs
* `output_suffix`: A suffix to add to the exported classifications corresponding to the model iteration

In [4]:
model_path = 'results/indian_ocean_ml_model_20220309.joblib'

training_data = "results/training_data/indian_ocean_training_data_20220309.txt"

test_shapefile = 'data/s2_tiles_indian_ocean.geojson'

results = 'results/classifications/'

output_suffix = '20220309'

dask_chunks = {'x':-1, 'y':-1}

### Open and inspect test_shapefile

In [5]:
gdf = gpd.read_file(test_shapefile)

In [6]:
# gdf.explore(column='title')
# gdf.head()
# map_shapefile(gdf, attribute='title')

## Open the model

The code below will also re-open the training data we exported from `3_Train_fit_evaluate_classifier.ipynb`

In [7]:
model = load(model_path).set_params(n_jobs=1)

In [8]:
# load the data
model_input = np.loadtxt(training_data)

# load the column_names
with open(training_data, 'r') as file:
    header = file.readline()
    
column_names = header.split()[1:][1:]

## Making a prediction


### Set up datacube query

These query options should exactl match the query params in `1_Extract_training_data.ipynb`, unless there are measurements that no longer need to be loaded because they were dropped during a feature selection process (which we didn't conduct).

### Loop through tiles and predict, with post-processing

For every tile we list in the `test_shapefile`, we calculate the feature layers, and then use the DE Africa function `predict_xr` to classify the data.

The `feature_layer_functions.gm_mads_two_seasons_prediction` function is doing most of the heavy-lifting here.

The results are exported to file as Cloud-Optimised Geotiffs.

In [9]:
#make a folder to store results in if one doesn't already exist
if not os.path.exists(results+output_suffix):
        os.mkdir(results+output_suffix)

In [10]:
import gc
g0, g1, g2 = gc.get_threshold()
gc.set_threshold(g0*3, g1*3, g2*3)

In [11]:
#get affine of all tiles as geobox object (at 20m res)
geobox_dict = AfricaGeobox(resolution=20)

def cropmask(index,row,gdf,model,column_names,results,output_suffix):
    
    #grab x,y id from the geopan
    x,y = gdf.loc[index]['title'].split(",")
    x, y = int(x)+181, int(y)+77
    
    if os.path.isfile(results+output_suffix+'/Indian_ocean_'+str(x)+'_'+str(y)+'_prediction_pixel_'+output_suffix+'.tif'):
        pass
    else:
        #get affine of single tile as geobox object
        geobox = geobox_dict[(x,y)]

        #run feature layer function
        data = gm_mads_two_seasons_prediction(geobox,dask_chunks=dask_chunks)
        data = data[column_names]

        #predict using the imported model
        with HiddenPrints():
            predicted = predict_xr(model,
                               data,
                               clean=True,
                              ).compute()

        #-------Post-processsing ------------------------------       
        predict = post_processing(predicted)
        
        #----export classifications to disk-----------------------
        write_cog(predict,
                  results+output_suffix+'/Indian_ocean_'+str(x)+'_'+str(y)+'_prediction_pixel_'+output_suffix+'.tif',
                  overwrite=True)


### Simple for loop for testing

In [12]:
%%time
i=1
for index, row in gdf.iterrows():
    print('working on tile: '+str(i)+"/"+str(len(gdf)),end='\r')
    cropmask(index,row,gdf,model,column_names,results,output_suffix)
    i+=1

KeyboardInterrupt: 

/env/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/env/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: divide by zero encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/env/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))
/env/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in log
  return func(*(_execute_task(a, cache) for a in args))
/env/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/env/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: divide by zero encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/env/lib/python3.8/site-packa

### Plotting

In [ ]:
# import xarray as xr
# predict=xr.open_rasterio('results/classifications/20211022/indian_ocean_214_54_prediction_pixel_20211022.tif').squeeze()

In [ ]:
# predict.plot.imshow(size=12);

## Next steps

1. [Extracting_training_data](1_Extracting_training_data.ipynb) 
2. [Inspect_training_data](2_Inspect_training_data.ipynb)
3. [Train_fit_evaluate_classifier](3_Train_fit_evaluate_classifier.ipynb)
4. **Predict (this notebook)**


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

